In [1]:
from mcts_model import AlphaZeroNN, ReplayBuffer, MCTS_Policy, Trainer, generate_lbfs_start_states, CONFIG
from mcts_worker import run_episode_worker

In [2]:
lbfs_start_states = generate_lbfs_start_states(num_states=20000, warmup=100000, separation=5000)

--- Generating 20000 Warm Start States using LBFS ---
Warming up for 100000 time units...
  Captured 50/20000 states...
  Captured 100/20000 states...
  Captured 150/20000 states...
  Captured 200/20000 states...
  Captured 250/20000 states...
  Captured 300/20000 states...
  Captured 350/20000 states...
  Captured 400/20000 states...
  Captured 450/20000 states...
  Captured 500/20000 states...
  Captured 550/20000 states...
  Captured 600/20000 states...
  Captured 650/20000 states...
  Captured 700/20000 states...
  Captured 750/20000 states...
  Captured 800/20000 states...
  Captured 850/20000 states...
  Captured 900/20000 states...
  Captured 950/20000 states...
  Captured 1000/20000 states...
  Captured 1050/20000 states...
  Captured 1100/20000 states...
  Captured 1150/20000 states...
  Captured 1200/20000 states...
  Captured 1250/20000 states...
  Captured 1300/20000 states...
  Captured 1350/20000 states...
  Captured 1400/20000 states...
  Captured 1450/20000 states...
  

In [3]:
trainer = Trainer(CONFIG, model = start_states=lbfs_start_states)
trainer.run_training_loop_parallel(num_workers=4)

Trainer init: state_size=10, action_space_size=7

--- PARALLEL TRAINING: 100 loops x 50 episodes ---
Workers: 4

--- LOOP 1/100 ---
  > Episode 50/50 finished. (Last Score: -0.844, Size: 56.47)
  Generated 20628 samples.
  Avg Score: -0.5681
  Avg Sys Size: 32.4242
  Avg Loss: 1.5613

--- LOOP 2/100 ---
  > Episode 50/50 finished. (Last Score: -0.842, Size: 56.20)
  Generated 20645 samples.
  Avg Score: -0.5594
  Avg Sys Size: 31.8739
  Avg Loss: 1.3637

--- LOOP 3/100 ---
  > Episode 50/50 finished. (Last Score: -0.503, Size: 26.20)
  Generated 20631 samples.
  Avg Score: -0.6094
  Avg Sys Size: 36.3881
  Avg Loss: 1.3682

--- LOOP 4/100 ---


Process SpawnPoolWorker-13:
Process SpawnPoolWorker-15:
Process SpawnPoolWorker-16:
Process SpawnPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.12/3.12.8/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/Users/jesus/Desktop/research/mcts_worker.py", line 99, in run_episode_worker
    assignments, root, state_vec = local_policy.decide(net, 

KeyboardInterrupt: 

In [4]:
trainer.save_model()

Model saved to mcts_alphazero_model.pth
